In [31]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
from transformers import AutoTokenizer
from tqdm import tqdm
import scml
from scml import nlp as snlp
import mylib

In [32]:
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [33]:
pretrained_dir = "pretrained/google/electra-small-discriminator"
model_max_length = 512
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=model_max_length)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [34]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661 entries, 0 to 19660
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             19661 non-null  object
 1   pub_title      19661 non-null  object
 2   dataset_title  19661 non-null  object
 3   dataset_label  19661 non-null  object
 4   cleaned_label  19661 non-null  object
dtypes: object(5)
memory usage: 768.1+ KB


In [35]:
gt_map: Dict[str, List[str]] = {}
dl_map: Dict[str, List[str]] = {}
for t in tqdm(train.itertuples()):
    pid = getattr(t, "Id")
    cl = getattr(t, "cleaned_label")
    dl = getattr(t, "dataset_label")
    if pid not in gt_map:
        gt_map[pid] = list()
    if pid not in dl_map:
        dl_map[pid] = list()
    gt_map[pid].append(cl)
    dl_map[pid].append(dl)

19661it [00:00, 330459.60it/s]


In [36]:
rows = []
sep = "|"
m = 650
negative_examples = 3
max_length = 500
for rid, gts in tqdm(gt_map.items()):
    is_multi = 0
    if len(gts) > 1:
        is_multi = 1
    ground_truth = sep.join(sorted(gts))
    dataset_labels = sep.join(sorted(dl_map[rid]))
    with open(f"input/train/{rid}.json") as in_file:
        sections = json.load(in_file)
    tmp = []
    for section in sections:
        tmp.append(section["text"])
    text = " ".join(tmp).strip()
    text = snlp.to_ascii_str(text)
    spans = []
    for dl in dl_map[rid]:
        i = 0
        j = len(text)
        while i < j - len(dl) + 1:
            a = text.find(dl, i, j)
            if a == -1:
                break
            b = a + len(dl)
            spans.append((a, b))
            i = b
            # Right context: answer is on the left edge of window
            b = 2 * m + a
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                b -= 100
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_start"] = 0
            row["answer_end"] = len(dl)
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)

100%|██████████| 14316/14316 [00:51<00:00, 279.08it/s]


In [37]:
df = pd.DataFrame.from_records(rows)
#df = pd.DataFrame(tmp, columns=["Id", "is_multi", "ground_truth", "dataset_labels"])
cols = ["is_multi", "is_impossible"]
df[cols] = df[cols].astype(np.int8)
cols = ["answer_start", "answer_end", "context_token_length"]
df[cols] = df[cols].astype(np.int8) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61002 entries, 0 to 61001
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Id                    61002 non-null  object
 1   is_multi              61002 non-null  int8  
 2   ground_truth          61002 non-null  object
 3   dataset_labels        61002 non-null  object
 4   is_impossible         61002 non-null  int8  
 5   answer_start          61002 non-null  int8  
 6   answer_end            61002 non-null  int8  
 7   context               61002 non-null  object
 8   context_token_length  61002 non-null  int8  
dtypes: int8(5), object(4)
memory usage: 2.2+ MB


In [38]:
df.sample(20).head(20)

,Id,is_multi,ground_truth,dataset_labels,is_impossible,answer_start,answer_end,context,context_token_length
27897,50360796-e02b-49e1-8de8-4f9c01f715f2,1,adni|alzheimer s disease neuroimaging initiative adni,ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI),0,0,4,"ADNI subjects (mean ages 80 years 11 months vs. 75 years 5 months). Fourth, the ADNI MCI sample was restricted to the AD prodrome, a-MCI, and the AD sample was restricted to probable AD. The Wechsler logical memory test was used to distinguish a-MCI subjects from other MCI subjects during initial testing: Subjects were required to have scores in the 2-8 range on the delayed part of this test (the maximum score is 25) to be included in the MCI sample. As a result, although ADAMS MCIs were a mixture of a-MCI and n-MCI, ADNI MCIs were exclusively a-MCI. This is important when it comes to testing theoretical hypotheses about the retrieval processes that differentiate HC, MCI, and AD groups and that predict future HC MCI and MCI AD transitions. Those processes might be different when the MCI group is restricted to subjects with significant memory impairment and when the AD group is restricted to probable AD.\nAs mentioned, ADNI subjects were rediagnosed at regular intervals over the next 2 years. Nearly half the a-MCI subjects converted to AD, and the rest did not (designated as the a-MCI C and a-MCI NC subgroups below). A small proportion (8%) of the HC subjects converted to a-MCI, and the rest did not (designated as the HC C and HC NC subgroups below). Before detailed results are",55
39706,7bba3169-50ae-4ac7-9c5a-b8dba5918149,0,adni,ADNI,0,0,4,"ADNI study to ensure compatibility across scanners [28] . Full brain and skull coverage was required and detailed quality control was carried out on all MR images according to the AddNeuroMed quality control procedure [23, 29] . We applied the Freesurfer pipeline (version 4.5.0) to the MRI images to produce regional cortical thickness and subcortical volumetric measures. Cortical reconstruction and subcortical volumetric segmentation includes removal of non-brain tissue using a hybrid watershed/surface deformation procedure [30] , automated Talairach transformation, segmentation of the subcortical white matter and deep grey matter volumetric structures (including hippocampus, amygdala, caudate, putamen, ventricles) [30] [31] [32] , intensity normalization [33] , tessellation of the grey matter white matter boundary, automated topology correction [34, 35] , and surface deformation following intensity gradients to optimally place the grey/white and grey/cerebrospinal fluid borders at the location where the greatest shift in intensity defines the transition to the other tissue class [36] [37] [38] . Once the cortical models are complete, registration to a spherical atlas takes place which utilizes individual cortical folding patterns to match cortical geometry across subjects [39] .",28
11450,3f3033e9-4f55-4fd0-ac47-330be5a51791,1,adni|alzheimer s disease neuroimaging initiative adni,ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI),0,0,4,"ADNI, and also one of the most widely used test of dementia severity in NACC [30] .\nDepression scores (Geriatric Depression Scale, GDS): These are scores from a screening test designed to identify depression symptoms in the elderly. A 15-item GDS bigger or equal to 6 was an exclusion criterion in ADNI. Individuals with history of major depression or bipolar disorder within the past year were also excluded from the study. In NACC, GDS is not recorded for individuals with severe dementia, but it is not used as an exclusion criterion for study entry. It should be noted, however, that GDS is not a required form in NACC, so it may not be recorded for any participant at any visit. Due to the inclusion/exclusion criteria and the reduced number of assessments of the GDS score in the datasets considered, the results and conclusions about the potential effect of depression should be interpreted 

In [41]:
df["is_multi"].value_counts(normalize=True)

1    0.663519
0    0.336481
Name: is_multi, dtype: float64

In [40]:
df.to_parquet("output/train.parquet", index=False)